# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils


import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [ ]:
reduce_y_var_lags = True
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = planning_and_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

In [ ]:
stop!

# PGAM (not used anymore because of importing issue after I made a new venv & higher version of python)

Code is from https://github.com/BalzaniEdoardo/PGAM

I might need to run this on google colab ...? Or, look into clusters? or maybe mac pro is enough?

In [ ]:
if not '/Users/dusiyi/Documents/Multifirefly-Project/PGAM/src/PGAM' in sys.path: 
    sys.path.append('/Users/dusiyi/Documents/Multifirefly-Project/PGAM/src/PGAM')
from GAM_library import *
import gam_data_handlers as gdh
from post_processing import postprocess_results
from scipy.io import savemat

## individual steps

### categorize variables

In [ ]:
pgam_inst = pgam_class.PGAMclass(data_item.x_var, data_item.y_var, data_item.bin_width, data_item.processed_neural_data_folder_path)

In [ ]:
pgam_inst.prepare_for_pgam(num_total_trials=10)

### temporal kernel

modified from PGAM_Tutorial.ipynb

In [ ]:
pgam_inst._add_temporal_features_to_model()

In [ ]:
help(gdh.smooths_handler.add_smooth)

### spatial variable

In [ ]:
pgam_inst._add_spatial_features_to_model()

### run

In [ ]:
pgam_inst.run_pgam(neural_cluster_number=10)

### post-processing

In [ ]:
pgam_inst.post_processing()

### save results

In [ ]:
pgam_inst.save_results()

In [ ]:
stop!

## iterate through all neurons

In [ ]:
pgam_inst = pgam_class.PGAMclass(data_item.x_var, data_item.y_var, data_item.bin_width, data_item.processed_neural_data_folder_path)

In [ ]:
for i in range(data_item.x_var.shape[1]):
    print(f'neural_cluster_number: {i} out of {data_item.x_var.shape[1]}')
    pgam_inst.streamline_pgam(neural_cluster_number=i, num_total_trials=10)